In [1]:

import numpy as np
import matplotlib.pyplot as plt
# import cvxpy as cp 
from scipy import optimize
from scipy.linalg import toeplitz
from numpy import fft 
from core.core import *
from util.detector import * 
from util.estimator import * 
from util.modem import * 

import random
import os 

### The Maximum Likelihood detection

In [3]:
def ml(y_re, H_re, symbol_space, snr_dB):
    """
    Maximum Likelihood detection for KxN mu-MIMO channel (uses Gaussian distribution) 
    """
    # complex to real and then reshape for broadcast
    snr = 10**(snr_dB/20)
    T = H_re.shape[0]
    K = H_re.shape[-1] // 2
    M = int(symbol_space.shape[0] ** (1/K))
    x_shape = tuple([T, K, 1])
    symbol_space_re = np.hstack((symbol_space.real, symbol_space.imag)) * np.ones(shape=(T, M**K, 2*K))
    likelihood = phi(np.matmul(symbol_space_re, H_re.transpose(0, 2, 1)) * y_re.transpose(0, 2, 1) * np.sqrt(2 * snr))
    log_likelihood = np.sum(np.log(likelihood), axis=2)
    x_hat_idx = np.argmax(log_likelihood, axis=1)
    x_hat_ML = np.take(symbol_space, x_hat_idx, axis=0)
    x_hat_ML = x_hat_ML.reshape(x_shape)
    return x_hat_ML 

In [6]:

K = 2   # Number of users (transmitter antenna) 
N = 4   # Number of Receiver (Receive antenna)
M = 4   # level of modulation (4-QAM / QPSK)
T = 10 # Total Transmission rounds 
SNR_max = 30


def make_symbol_space(K, M, cp):
    """
    Makes symbol space S = |M|^K 
    :params: 
    :   K (int) : number of users\n
    :   M (int) : level of modulation\n
    :returns:
    :   symbol_space (np.ndarray) : symbol space S=|M|^K
    """
    index_space = np.indices([M for _ in range(K)]).reshape(K, -1).T 
    symbol_space = np.take(cp, index_space)
    return symbol_space 

def comp2re(H, x, z, y):
    """
    Real representation of complex matrix \n
    \t x = [x_r + j x_i] --->  x_re = [x_re x_im] \n
    \t H_re = \n \t \t[H_real, -H_imag \n
        \t H_imag, H_real]\n
    
    """
    x_re = np.hstack((x.real, x.imag))
    z_re = np.hstack((z.real, x.imag))
    y_re = np.hstack((y.real, y.imag)) 
    H_re = np.block([
        [H.real, -H.imag],
        [H.imag, H.real]
    ])
    return H_re, x_re, z_re, y_re 

snr = 5

# simple constellation points (normalized)
constellation_points = np.array([-1-1j, -1+1j, 1-1j, 1+1j])
constellation_points /= np.sqrt(2) 

# make symbol space = |M|^K  in case of using ML detection 
symbol_space = make_symbol_space(K, M, cp=constellation_points) 

x = np.random.randint(0, M, size=K * T).reshape(T, K, 1) 
x = np.take(constellation_points, x)

# Channel 
H = (1 / np.sqrt(2)) * (np.random.randn(T, N, K) + 1j * np.random.randn(T, N, K))
z = (1 / np.sqrt(2)) * (np.random.randn(T, N, 1) + 1j * np.random.randn(T, N, 1))

r = np.matmul(H, x) + (10**(-snr/20)) * z  

# 1-bit ADC 
y = np.sign(r.real) + 1j * np.sign(r.imag) 

H_re, x_re, z_re, y_re = comp2re(H, x, z, y)

In [7]:
x_re[0]

array([[0.70710678],
       [0.70710678],
       [0.70710678],
       [0.70710678]])

In [8]:
# H_re_tilde
H_re_tilde = H_re * y_re 
H_re_tilde.shape

(10, 8, 4)

In [16]:
rho = 5

nominator = np.exp(-rho * np.square((H_re_tilde @ x_tilde)))
denominator = phi(np.sqrt(2 * rho) * (H_re_tilde @ x_re))
grad = (1/np.sqrt(2 * np.pi)) * (nominator / denominator) 
def compute_grad(x_tilde):
    rho = 5 
    nominator = np.exp(-rho * np.square((H_re_tilde @ x_tilde)))
    denominator = phi(np.sqrt(2 * rho) * (H_re_tilde @ x_tilde))
    grad = (1 / np.sqrt(2 * np.pi)) * (nominator / denominator) 
    return grad 


NameError: name 'x_tilde' is not defined

### Near maximum Likelihood detection (Optimization method) 


In [12]:
# Initialize x_tilde for nML 
x_tilde = np.random.randn(T, 2*K, 1)

learning_rate = 0.01 

# for i in range(1):
#     x_tilde = x_tilde + learning_rate * (H_re_tilde.transpose(0, 2, 1) @ compute_grad(x_tilde=x_tilde))
    
#     if i % 10 == 0 : 
#         print()

In [13]:
x_tilde = np.random.randn(T, 2*K, 1) 

In [14]:
print(x_tilde[0])
snr_db = 5
snr = 10 ** (snr_db/20) 

denominator = phi(np.matmul(H_re_tilde, x_tilde) * np.sqrt(2 * snr))
nominator = np.exp(-snr * (np.matmul(H_re_tilde, x_tilde))**2)
grad = (1/np.sqrt(2 * np.pi)) * (nominator / denominator)
compute_grad = np.matmul(H_re_tilde.transpose(0, 2, 1), grad)

x_tilde = x_tilde + learning_rate * compute_grad
print(x_tilde[0])

def compute_grad(H_re_tilde, x_init, snr):
    denominator = phi(np.matmul(H_re_tilde, x_init) * np.sqrt(2 * snr))
    nominator = np.exp(-snr * (np.matmul(H_re_tilde, x_init))**2)
    grad = (1/np.sqrt(2 * np.pi)) * (nominator/denominator) 
    return np.matmul(H_re_tilde.transpose(0, 2, 1), grad)

def nML(y_re, H_re, x_init, symbol_space, snr_dB, iter=10):
    snr = 10 ** (snr_dB/20) 
    H_re_tilde = H_re * y_re 
    
    for _ in range(iter):
        x_init = compute_grad(H_re_tilde=H_re_tilde, x_init=x_init, snr=snr)
        print(x_init)

    return x_init



[[-1.39285429]
 [-0.6926307 ]
 [ 0.54805318]
 [ 1.37444169]]
[[-1.28541176]
 [-0.62557007]
 [ 0.5544774 ]
 [ 1.28502659]]


/var/folders/qf/g6m74fsx51j_2q1__9zjd_5m0000gn/T/ipykernel_1705/553622745.py:7: RuntimeWarning: divide by zero encountered in true_divide
  grad = (1/np.sqrt(2 * np.pi)) * (nominator / denominator)


In [20]:
x_tilde = np.random.randn(T, 2*K, 1)
snr_db = 0

x_tilde = nML(y_re, H_re, x_tilde, symbol_space=constellation_points, iter=10, snr_dB=snr_db)

[[[ -4.9021208 ]
  [  2.97781715]
  [  0.54081234]
  [  1.93664872]]

 [[ -5.58238847]
  [-11.2407415 ]
  [  0.50241107]
  [  0.27478379]]

 [[  0.05424629]
  [ -2.83753068]
  [ -0.79904909]
  [  7.63293941]]

 [[  2.35102759]
  [ 13.10249812]
  [  0.26751112]
  [ -0.14752874]]

 [[  4.82322682]
  [  3.39998119]
  [  0.48708919]
  [ -2.73321978]]

 [[ -7.36936713]
  [  0.61845364]
  [  5.45413364]
  [ -2.71360301]]

 [[  1.38467795]
  [ -0.07304854]
  [ -1.62276345]
  [  2.1091581 ]]

 [[  2.02765342]
  [ -0.33881522]
  [  1.63473925]
  [ -1.15697542]]

 [[ -1.34705202]
  [  6.55914472]
  [ -0.48944353]
  [  7.86266614]]

 [[  4.68841785]
  [ -8.64550081]
  [  2.41471034]
  [  1.22037849]]]
[[[ -1.22336758]
  [ -5.07611319]
  [  1.65345881]
  [  1.24849486]]

 [[  3.10318831]
  [  1.2976264 ]
  [ -5.42158032]
  [ -4.21268014]]

 [[ -1.09049527]
  [  0.42695784]
  [ -0.40217186]
  [  0.06749828]]

 [[  0.78797981]
  [ -0.30644474]
  [ -1.77288561]
  [  1.81673355]]

 [[        -inf]
  [

/Users/lab2020/opt/anaconda3/envs/learning/lib/python3.7/site-packages/ipykernel_launcher.py:16: RuntimeWarning: divide by zero encountered in true_divide
  app.launch_new_instance()
/Users/lab2020/opt/anaconda3/envs/learning/lib/python3.7/site-packages/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in matmul
  
/Users/lab2020/opt/anaconda3/envs/learning/lib/python3.7/site-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in matmul
  from ipykernel import kernelapp as app
/Users/lab2020/opt/anaconda3/envs/learning/lib/python3.7/site-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in true_divide
  app.launch_new_instance()


In [15]:
x= x_re[0] 
H = H_re[0] 
y = y_re[0]


In [16]:
x.shape, H.shape, y.shape

((4, 1), (8, 4), (8, 1))

In [17]:
x_init = np.random.randn(4, 1)

In [19]:
sers_avg = []
snrs = []
err = 0

In [55]:
def cost(G_tilde, x_init, snr):
    snr = 10 ** (snr/20) 
    g = np.sqrt(2 * snr) * (G_tilde @ x_init) 
    g = phi(g) 
    g = np.log(g) 
    g = np.sum(g) 
    return g

def compute_gradient(G_tilde, x_init, snr):
    snr = 10 ** (snr/20) 
    inner = G_tilde @ x_init 
    print("inner product term:", inner)
    nom = inner ** 2
    nom = np.exp(-snr * nom) 
    print("numerator", nom)
    denom = np.sqrt(2 * snr) * inner 
    denom = phi(denom)
    print("denominator: ", denom)
    grad = (1 / np.sqrt(2 * np.pi)) * (nom / denom) 
    grad = G_tilde.T @ grad 
    return grad

def GD(x_init, snr, epoch=100, lr=0.01):
    global G_tilde 
    costs = []
    for e in range(epoch):
        x_init = x_init + lr * compute_gradient(G_tilde=G_tilde, x_init=x_init, snr=snr)
        if np.linalg.norm(x_init) >= np.sqrt(K):
            x_init /= np.linalg.norm(x_init)
            x_init *= np.sqrt(K)
        costs.append(cost(G_tilde=G_tilde, x_init=x_init, snr=snr))
        if e % 10 == 0 : 
            print("cost=", cost(G_tilde=G_tilde, x_init=x_init, snr=snr))
    return costs, x_init

def comp2re(H, x, z, y):
    x_re = np.vstack((x.real, x.imag))
    z_re = np.vstack((z.real, x.imag))
    y_re = np.vstack((y.real, y.imag)) 
    H_re = np.block([
        [H.real, -H.imag],
        [H.imag, H.real]
    ])
    return H_re, x_re, z_re, y_re 

# ser_avg = 0 
# trial = 10
# err = 0

# K = 2; N = 4; M = 4; snr=20
# constellation_points = np.array([-1-1j, -1+1j, 1-1j, 1+1j])
# constellation_points /= np.sqrt(2) 

# x = np.random.randint(0, M, size=K).reshape(K, 1) 
# x = np.take(constellation_points, x)
# # Channel 
# H = (1 / np.sqrt(2)) * (np.random.randn(N, K) + 1j * np.random.randn(N, K))
# z = (1 / np.sqrt(2)) * (np.random.randn(N, 1) + 1j * np.random.randn(N, 1))

# r = np.matmul(H, x) + (10**(-snr/20)) * z  

# # 1-bit ADC 
# y = np.sign(r.real) + 1j * np.sign(r.imag) 

# H_re, x_re, z_re, y_re = comp2re(H, x, z, y)

# G_tilde = H_re * y_re 

# x_init = np.random.randn(2*K, 1) 
# x_init = x_init / np.linalg.norm(x_init)
# _, x_tilde_re = GD(x_init=x_init, snr=snr, epoch=100, lr=0.01)


# x_tilde = x_tilde_re[:K] + 1j * x_tilde_re[K:]
# x_det_first_stage = symbol_by_symbol(x_tilde=x_tilde, constellation_points=constellation_points)
# print("detected: ", x_det_first_stage.squeeze(), "label:", x.squeeze())
# err += (K - np.sum(np.isclose(x_det_first_stage, x)))
# print(err)
# # err += (K - np.sum(np.isclose(x_det_first_stage, x)))

# # print(err)

In [64]:
test = np.exp(-(10**(snr/20))*(-1.949)**2)
test = np.sqrt(2 * (10**(0/20))) * -1.949
test = phi(test)
print(test)

0.002922947624310668


In [67]:
GD(x_init=x_init, snr=1, epoch=100, lr=0.01)

inner product term: [[-0.19563831]
 [ 0.44242915]
 [-1.9493769 ]
 [-0.69808852]
 [-0.51312635]
 [-0.78845971]
 [-1.05735653]
 [-0.22458081]]
numerator [[0.95796453]
 [0.80281748]
 [0.01406946]
 [0.57880448]
 [0.74421453]
 [0.49781649]
 [0.2852413 ]
 [0.94498075]]
denominator:  [[0.38473496]
 [0.74625906]
 [0.0017491 ]
 [0.14783956]
 [0.22104495]
 [0.11877727]
 [0.05660463]
 [0.36827538]]
cost= -14.402207791731316
inner product term: [[-0.14351316]
 [ 0.47931024]
 [-1.68269774]
 [-0.59581145]
 [-0.41459199]
 [-0.64429602]
 [-0.88247907]
 [-0.19130747]]
numerator [[0.97715585]
 [0.77277247]
 [0.04171212]
 [0.67145601]
 [0.82459754]
 [0.62765204]
 [0.41736525]
 [0.95976747]]
denominator:  [[0.41488978]
 [0.76362506]
 [0.00585609]
 [0.1860539 ]
 [0.26727897]
 [0.16723171]
 [0.09309018]
 [0.3872167 ]]
inner product term: [[-0.0970127 ]
 [ 0.51348469]
 [-1.44436824]
 [-0.50424172]
 [-0.32592349]
 [-0.51515046]
 [-0.72472409]
 [-0.16113348]]
numerator [[0.98949572]
 [0.7439074 ]
 [0.09625499]

([-14.402207791731316,
  -12.296026079763848,
  -10.59150704836453,
  -9.204191644455365,
  -8.068285851374654,
  -7.132435874664075,
  -6.356466674718757,
  -5.708864869492898,
  -5.164836420978259,
  -4.704807838601627,
  -4.313269359478618,
  -3.977881625414508,
  -3.6887852624082242,
  -3.4380666350166207,
  -3.219343788746179,
  -3.027444901557717,
  -2.8581579799351697,
  -2.7080354771006965,
  -2.574241310727558,
  -2.4544306732686545,
  -2.346655261809676,
  -2.2492882636598006,
  -2.160964740984527,
  -2.080534057183035,
  -2.007021752165607,
  -1.93959885897078,
  -1.8775571028991829,
  -1.8202887690175953,
  -1.7672702892178398,
  -1.7180488047824274,
  -1.6722311188789452,
  -1.6294745764165817,
  -1.5894795044912096,
  -1.5519829214926353,
  -1.5167532816312204,
  -1.4835860678147033,
  -1.4523000822670649,
  -1.4227343131772152,
  -1.3947452786491388,
  -1.368204767573018,
  -1.3429979117369268,
  -1.3190215353185397,
  -1.2961827374340378,
  -1.2743976711447416,
  -1.253

In [344]:
x_tilde = x_tilde_re[:K] + 1j * x_tilde_re[K:]
x_det_first_stage = symbol_by_symbol(x_tilde=x_tilde, constellation_points=constellation_points)
x_det_first_stage, x[:K] + 1j * x[K:]

(array([[-0.70710678-0.70710678j],
        [-0.70710678-0.70710678j]]),
 array([[-0.70710678+0.70710678j],
        [ 0.70710678+0.70710678j]]))

In [ ]:
K = 4   # Number of users (transmitter antenna) 
N = 16   # Number of Receiver (Receive antenna)
M = 4   # level of modulation (4-QAM / QPSK)
T = 10 # Total Transmission rounds 
SNR_max = 30


def make_symbol_space(K, M, cp):
    """
    Makes symbol space S = |M|^K 
    :params: 
    :   K (int) : number of users\n
    :   M (int) : level of modulation\n
    :returns:
    :   symbol_space (np.ndarray) : symbol space S=|M|^K
    """
    index_space = np.indices([M for _ in range(K)]).reshape(K, -1).T 
    symbol_space = np.take(cp, index_space)
    return symbol_space 

def comp2re(H, x, z, y):
    """
    Real representation of complex matrix \n
    \t x = [x_r + j x_i] --->  x_re = [x_re x_im] \n
    \t H_re = \n \t \t[H_real, -H_imag \n
        \t H_imag, H_real]\n
    
    """
    x_re = np.hstack((x.real, x.imag))
    z_re = np.hstack((z.real, x.imag))
    y_re = np.hstack((y.real, y.imag)) 
    H_re = np.block([
        [H.real, -H.imag],
        [H.imag, H.real]
    ])
    return H_re, x_re, z_re, y_re 

snr = 5

# simple constellation points (normalized)
constellation_points = np.array([-1-1j, -1+1j, 1-1j, 1+1j])
constellation_points /= np.sqrt(2) 

# make symbol space = |M|^K  in case of using ML detection 
symbol_space = make_symbol_space(K, M, cp=constellation_points) 

x = np.random.randint(0, M, size=K * T).reshape(T, K, 1) 
x = np.take(constellation_points, x)

# Channel 
H = (1 / np.sqrt(2)) * (np.random.randn(T, N, K) + 1j * np.random.randn(T, N, K))
z = (1 / np.sqrt(2)) * (np.random.randn(T, N, 1) + 1j * np.random.randn(T, N, 1))

r = np.matmul(H, x) + (10**(-snr/20)) * z  

# 1-bit ADC 
y = np.sign(r.real) + 1j * np.sign(r.imag) 

H_re, x_re, z_re, y_re = comp2re(H, x, z, y)